## TLS in the web browser in gory detail

### Cipher and connection negotiation
The TLS handshake starts with the client connecting to the server with plaintext cipher and connection details negotiation. The server and the client then go back and forth negotiating the connection details until the server is satisfied that all necessary choices have been made.

### Certificate checking
Once the server reaches this point, it sends the client its public certificate. The client requests the intermediate certificate linked by to this certificate from the network. It then verifies that the authority behind that certificate actually issued the origin certificate by decrypting the signature of the origin certificate with the public key on the intermediate certificate. If the decrypted value matches the ID (the domain name) of the origin certificate, it then goes one layer higher, requesting the next intermediate certificate.

This process is continued until the client reaches the top-level certificate. The top-level certificate is self-signed: its signature is its own public key applied to its ID. However, the top-level certificate is hard-coded into our client as belonging to a trusted certificate authority, so we trust this certificate intrinsically.

If we trust the top-level certificate, then by the transfer property we trust the entire chain of certificates. Only the private key of the top-level certificate authority could have been used to generate the signature of the next-highest-level certificate; e.g. we have a signature-based chain of trust that descends down to the origin certificate.

This verifies identity: we now know that the domain we are connected to is who it purports to be.

Attacks against signatures are limited to the interception or decryption of the private key.

### Symmetric key negotiation
The client now generates a random symmetric secret key, encrypts it with the public key, and sends it to the server. Only the server may decrypt this value, with its private key, at which point the symmetric secret key is a shared secret. To verify that the server recieved the symmetric private key, the server sends a "Finished" response encoded with the symmetric key to the client, which the client decrypts and checks for equivalence to the expected value.

A message authorization code is used to guard against message corruption and tampering. The message authorization code is a function of the data payload that is itself encrypted (with the public key, in the case of the message containing the symmetric key; with the symmetric key, later on). When the client sends data to the server, the server verifies that the data payload has not been tampered with or degraded in flight by decrypting the message authorization code and verifying that it is equivalent to the hash of the data payload.

This design is vulnerable to a man-in-the-middle attack. A man in the middle may take the server-bound symmetric key message and substitute its own public key encrypted symmetric key. The server receives the key, thinks it to be from the client, decrypts it, sets it as its shared secret, and sends back the encrypted expected value. The man in the middle intercepts this message. The client never recieves the message, breaking the connection. The server now shares the symmetric key secret with the man in the middle.

As a result, after successful TLS negotiation the client knows authoritatively that it is speaking to the server, but the reverse is not true. The MAC does not protect against a man in the middle attack that intercepts the initial symmetric key message. Verifying the identity of the client once the symmetric key is set is the responsibility of the application layer (in the case of most web applications), or the responsibility of two-way TLS (in the case of some business-to-business or internal applications).